## Convolutional NN to classify govuk content to level2 taxons

Based on:
https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

### Load requirements and data

In [20]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import (Embedding, Input, Dense, 
                          Activation, Conv1D, MaxPooling1D, Flatten)
from keras.models import Model, Sequential
from keras.optimizers import rmsprop
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer


In [2]:
!which python3

/Users/ellieking/Documents/tag_tax/govuk-taxonomy-supervised-learning/tax_SL/bin/python3


In [3]:
labelled_level2 = pd.read_csv('../../../data/labelled_level2.csv', dtype=object)

## Hyperparameters

In [4]:
#MAX_NB_WORDS
MAX_SEQUENCE_LENGTH =1000
#EMBEDDING_DIM

### Create target/Y 

Note: when using the categorical_crossentropy loss, your targets should be in categorical format (e.g. if you have 10 classes, the target for each sample should be a 10-dimensional vector that is all-zeros expect for a 1 at the index corresponding to the class of the sample). In order to convert integer targets into categorical targets, you can use the Keras utility to_categorical

In [26]:
labelled_level2['level2taxon'] = labelled_level2['level2taxon'].astype('category')

labels = labelled_level2['level2taxon'].cat.codes 


In multilabel learning, the joint set of binary classification tasks is expressed with label binary indicator array: each sample is one row of a 2d array of shape (n_samples, n_classes) with binary values:  
the one, i.e. the non zero elements, corresponds to the subset of labels.  
An array such as np.array([[1, 0, 0], [0, 1, 1], [0, 0, 0]]) represents label 0 in the first sample, labels 1 and 2 in the second sample, and no labels in the third sample.  
Producing multilabel data as a list of sets of labels may be more intuitive. The MultiLabelBinarizer transformer can be used to convert between a collection of collections of labels and the indicator format.

####  first reshape wide to get columns for each level2taxon and row number = number unique urls

In [153]:
#get a smaller copy of data for pivoting ease (think you can work from full data actually and other cols get droopedauto)

level2_reduced = labelled_level2[['content_id', 'level2taxon', 'combined_text']].copy()

#how many level2taxons are there?
print(level2_reduced.level2taxon.nunique())

#count the number of taxons per content item into new column
level2_reduced['num_taxon_per_content'] = level2_reduced.groupby(["content_id"])['content_id'].transform("count")

#Add 1 because of zero-indexing to get 1-number of level2taxons as numerical targets
level2_reduced['level2taxon_code'] = level2_reduced.level2taxon.astype('category').cat.codes + 1

440


In [155]:
#reshape to wide per taxon and keep the combined text so indexing is consistent when splitting X from Y

multilabel = (level2_reduced.pivot_table(index=['content_id', 'combined_text'], 
                  columns='level2taxon_code', 
                  values='num_taxon_per_content'))
print(level2_reduced.shape)
print(multilabel.shape)

list(multilabel.columns)

(173560, 5)
(114048, 440)


,level2taxon_code,1,2,3,4,5,6,7,8,9,10,...,431,432,433,434,435,436,437,438,439,440
content_id,combined_text,,,,,,,,,,,,,,,,,,,,,
00029fa4-9b60-4285-898c-85ae8a6367f5,emma jones - small business crown representative as small business crown representative emma is keen to help uk smes win government business. emma was appointed as small business crown representative in july 2016. she was selected for the role because of her wealth of experience in working with smes. she is the founder of small business support group enterprise nation and the co-founder of startup britain. emma’s work in her role as small business crown representative includes: working with government and the small business panel to identify the remaining barriers to smes doing business with the public sector supporting the launch and delivery of the campaign to help show that government is “open for business” for smes and helping them bid for and win more contracts increasing awareness among smaller businesses of opportunities to deliver on behalf of larger private sector firms who have secured government contracts working with government to identify new opportunities to get best value from smes getting support emma is keen to hear what small business have to say and wants to engage with as many smes as possible. so if you’re thinking about becoming a government supplier take a look at the events and opportunities below for how to get involved and gain support. events the leeds cross government sme roadshow - 24 november 2017 - is a great opportunity for smes to hear directly about the opportunities to sell to the public sector. more information about the event and how to register can be found here. webinars register for free for emma’s half-hour webinars offering advice on how to become a government supplier. a list of webinars coming up is featured below. blogs read emma’s blogs to gain useful insight updates and tips for smes and government buyers. these smes did it and so can you! prompt payment makes for good business 2017 a big year for small businesses calling central government buyers: emma can help you meet your target small business saturday dec 2016: top tips for selling to government selling to the public sector guide in partner with the crown commercial service emma has developed a guide for small businesses with tips on selling to government. read here . government is open for business ‘open for business’ is the government’s campaign to reach more smes as potential suppliers: to help and support them to become suppliers and to listen to how government can improve the process. for more information visit www.gov.uk/openforbusiness register with contracts finder to keep updated on new and upcoming contracts worth over £10 000. for inspiration on how other small business have grown and benefitted from being a supplier government read our case studies . if you would like to help in getting the message out that government is open for business then visit the resources page for ways in which you can support.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00037b70-5b08-44c2-bf0a-fa8eb636a60b,land remediation: bringing brownfield sites back to use brochure showing uk expertise in land remediation outlining technologies systems and ideas used in the regeneration of industrial land. the uk was the first industrialised country in the world. the legacy of the industrial revolution is over 400 000 hectares of contaminated land. uk expertise in land remediation has been borne out of necessity. the department for international trade’s ( dit ) brochure provides an overview of the expertise gained from over 5 decades of experience in land remediation. the brochure includes information on: sector specialists urban regeneration spill response monitoring and validation corporate liability management innovation industry bodies how dit can help this was published originally by uk trade and investment which has since moved to the departm

In [143]:
#convert the number_of_taxons_per_content values to 1, meaning there was an entry for this taxon and this content_id, 0 otherwise
binary_multilabel = multilabel.notnull().astype('int')

In [144]:
#will convert columns to an array of shape
binary_multilabel[list(binary_multilabel.columns)].values.shape

(114048, 440)

In [145]:
#convert columns to an array. Each row represents a content item, each column an individual taxon
binary_multilabel = binary_multilabel[list(binary_multilabel.columns)].values
binary_multilabel[2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,

In [149]:
type(binary_multilabel)

numpy.ndarray

In [146]:
# mlb = MultiLabelBinarizer()
# y = mlb.fit_transform(binary_multilabel)
# y.shape

In [5]:
#Use this for singlelabel problems
labels = to_categorical(np.asarray(labels))

print('Shape of label tensor:', labels.shape)

labels

Shape of label tensor: (173560, 440)


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

### Create language data/X

format our text samples and labels into tensors that can be fed into a neural network. To do this, we will rely on Keras utilities keras.preprocessing.text.Tokenizer and keras.preprocessing.sequence.pad_sequences.

In [161]:
multilabel.index.names
texts = multilabel.index.get_level_values('combined_text')
texts.shape

(114048,)

In [162]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts) #yield one sequence per input text

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen= MAX_SEQUENCE_LENGTH) #MAX_SEQUENCE_LENGTH

Found 213132 unique tokens.


In [163]:
print('Shape of label tensor:', binary_multilabel.shape)
print('Shape of data tensor:', data.shape)

Shape of label tensor: (114048, 440)
Shape of data tensor: (114048, 1000)


### Data split

In [164]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = binary_multilabel[indices]
nb_validation_samples = int(0.2 * data.shape[0]) #validation split
print('nb_validationsamples:', nb_validation_samples)

x_train = data[:-nb_validation_samples]
print('Shape of x_train:', x_train.shape)
y_train = labels[:-nb_validation_samples]
print('Shape of y_train:', y_train.shape)
x_val = data[-nb_validation_samples:]
print('Shape of x_val:', x_val.shape)
y_val = labels[-nb_validation_samples:]
print('Shape of y_val:', y_val.shape)

nb_validationsamples: 22809
Shape of x_train: (91239, 1000)
Shape of y_train: (91239, 440)
Shape of x_val: (22809, 1000)
Shape of y_val: (22809, 440)


### preparing the Embedding layer
compute an index mapping words ot known embeddings by parsing the data dump of pre-trained embeddings
NB stopwords haven't been removed yet...

In [165]:
embeddings_index = {}
f = open(os.path.join('../../../data/', 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


compute embedding matrix using embedding_index dict and word_index

In [166]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))# used 6B.100d.txt
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

load this embedding matrix into an Embedding layer. Note that we set trainable=False to prevent the weights from being updated during training.

In [167]:
embedding_layer = Embedding(len(word_index) + 1,
                            100, # used 6B.100d.txt
                            weights=[embedding_matrix],
                            input_length= MAX_SEQUENCE_LENGTH, #MAX_SEQUENCE LENGTH
                            trainable=False)

An Embedding layer should be fed sequences of integers, i.e. a 2D input of shape (samples, indices). These input sequences should be padded so that they all have the same length in a batch of input data (although an Embedding layer is capable of processing sequence of heterogenous length, if you don't pass an explicit input_length argument to the layer).

All that the Embedding layer does is to map the integer inputs to the vectors found at the corresponding index in the embedding matrix, i.e. the sequence [1, 2] would be converted to [embeddings[1], embeddings[2]]. This means that the output of the Embedding layer will be a 3D tensor of shape (samples, sequence_length, embedding_dim).

## Training a 1D convnet

In [168]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32') #MAX_SEQUENCE_LENGTH
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(x_train), activation='sigmoid')(x)

model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

Metric values are recorded at the end of each epoch on the training dataset. If a validation dataset is also provided, then the metric recorded is also calculated for the validation dataset.

All metrics are reported in verbose output and in the history object returned from calling the fit() function. In both cases, the name of the metric function is used as the key for the metric values. In the case of metrics for the validation dataset, the “val_” prefix is added to the key.

In [169]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 1000, 100)         21313300  
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 35, 128)           82048     
__________

In [170]:
logname = 'tf_logs/allgovuk_' + str(datetime.now())

tb = TensorBoard(
    log_dir=logname, histogram_freq=1, write_graph=True, write_images=True)

#tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=1, write_graph=True, write_images=True)

In [171]:
model.fit(
    x_train, y_train, 
    validation_data=(x_val, y_val), 
    epochs=4, batch_size=128, 
    callbacks=[tb]
)

ValueError: Error when checking target: expected dense_4 to have shape (None, 91239) but got array with shape (91239, 440)

Improve accuracy by training longer with some regularization mechanism (such as dropout) or by fine-tuning the Embedding layer.

We can also test how well we would have performed by not using pre-trained word embeddings, but instead initializing our Embedding layer from scratch and learning its weights during training. We just need to replace our Embedding layer with the following:

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH)
                            
After 2 epochs, this approach only gets us to 90% validation accuracy, less than what the previous model could reach in just one epoch. Our pre-trained embeddings were definitely buying us something. In general, using pre-trained embeddings is relevant for natural processing tasks were little training data is available (functionally the embeddings act as an injection of outside information which might prove useful for your model).

